<a href="https://colab.research.google.com/github/hcorbel/attention_rank_bert_stackoverflow/blob/main/Stackoverflow_attention_rank_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import sys
sys.path.append('/content/gdrive/MyDrive/attention_rank/P7')

In [4]:
"""Runs BERT over input data and writes out its attention maps to disk."""
!pip install tensorflow-addons
import argparse
import os
import numpy as np

import time
from bert import modeling_tf2
from bert import tokenization
import bpe_utils
import utils
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 10.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
class Example(object):
    """Represents a single input sequence to be passed into BERT."""

    def __init__(self, features, tokenizer, max_sequence_length, ):
        self.features = features

        if "tokens" in features:
            self.tokens = features["tokens"]
        else:
            if "text" in features:
                text = features["text"]
            else:
                text = " ".join(features["words"])
            self.tokens = ["[CLS]"] + tokenizer.tokenize(text) + ["[SEP]"]

        self.input_ids = tokenizer.convert_tokens_to_ids(self.tokens)
        self.segment_ids = [0] * len(self.tokens)
        self.input_mask = [1] * len(self.tokens)
        while len(self.input_ids) < max_sequence_length:
            self.input_ids.append(0)
            self.input_mask.append(0)
            self.segment_ids.append(0)


def examples_in_batches(examples, batch_size):
    for i in utils.logged_loop(range(1 + ((len(examples) - 1) // batch_size))):
        yield examples[i * batch_size:(i + 1) * batch_size]


class AttnMapExtractor(object):
    """Runs BERT over examples to get its attention maps."""

    def __init__(self, bert_config_file, init_checkpoint,
                 max_sequence_length=128, debug=False):
        make_placeholder = lambda name: tf.placeholder(
            tf.int32, shape=[None, max_sequence_length], name=name)
        self._input_ids = make_placeholder("input_ids")
        self._segment_ids = make_placeholder("segment_ids")
        self._input_mask = make_placeholder("input_mask")

        bert_config = modeling_tf2.BertConfig.from_json_file(bert_config_file)
        if debug:
            bert_config.num_hidden_layers = 3
            bert_config.hidden_size = 144
        self._attn_maps = modeling_tf2.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self._input_ids,
            input_mask=self._input_mask,
            token_type_ids=self._segment_ids,
            use_one_hot_embeddings=True).attn_maps

        if not debug:
            print("Loading BERT from checkpoint...")
            assignment_map, _ = modeling_tf2.get_assignment_map_from_checkpoint(
                tf.trainable_variables(), init_checkpoint)
            tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    def get_attn_maps(self, sess, examples):
        feed = {
            self._input_ids: np.vstack([e.input_ids for e in examples]),
            self._segment_ids: np.vstack([e.segment_ids for e in examples]),
            self._input_mask: np.vstack([e.input_mask for e in examples])
        }
        return sess.run(self._attn_maps, feed_dict=feed)

In [8]:
def main(process_position):
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("--cased", default=False, action='store_true', help="Don't lowercase the input.")
    parser.add_argument("--max_sequence_length", default=512, type=int, help="Maximum input sequence length after tokenization(default=128).")
    parser.add_argument("--batch_size", default=8, type=int, help="Batch size when running BERT (default=16).")
    parser.add_argument("--debug", default=False, action='store_true', help="Use tiny model for fast debugging.")
    parser.add_argument("--word_level", default=False, action='store_true', help="Get word-level rather than token-level attention.")
    args, unknown = parser.parse_known_args()
    extractor = AttnMapExtractor(os.path.join(bert_dir, "bert_config.json"),
                                    os.path.join(bert_dir, "bert_model.ckpt"),
                                    args.max_sequence_length, args.debug)

    print("Creating examples...")
    tokenizer = tokenization.FullTokenizer(vocab_file=os.path.join(bert_dir, "vocab.txt"), do_lower_case=not args.cased)

    for i in range(len(file_list)):

        preprocessed_data_file = save_path + file_list[i] + "_" + bert_name + ".json"

        examples = []
        for features in utils.load_json(preprocessed_data_file):
            # print(features)
            example = Example(features, tokenizer, args.max_sequence_length)
            # print(example.tokens)
            if len(example.input_ids) <= args.max_sequence_length:
                examples.append(example)

        print("processing file:", file_list[i], "pos:", process_position)
        run_time = time.time()
        print("running time", run_time - start_time)
        process_position += 1
        print("Extracting attention maps...")
        feature_dicts_with_attn = []
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for batch_of_examples in examples_in_batches(examples, args.batch_size):
                attns = extractor.get_attn_maps(sess, batch_of_examples)
                print(attns)
                for e, e_attn in zip(batch_of_examples, attns):
                    print(e, e_attn)
                    seq_len = len(e.tokens)
                    e.features["attns"] = e_attn[:, :, :seq_len, :seq_len].astype("float16")
                    e.features["tokens"] = e.tokens
                    print(e.tokens)
                    feature_dicts_with_attn.append(e.features)
        #print(np.shape(feature_dicts_with_attn[0]['attns']))  # (12, 12, 30, 30)
        #print(np.shape(feature_dicts_with_attn[1]['attns']))  # (12, 12, 23, 23) 23 = token'length
        if args.word_level:
            print("Converting to word-level attention...")
            bpe_utils.make_attn_word_level(
                feature_dicts_with_attn, tokenizer, args.cased)
        # it still a process of learn how to get familiar with a new keyboard.
        outpath = preprocessed_data_file.replace(".json", "")
        outpath += "_attn.pkl"
        print("Writing attention maps to {:}...".format(outpath))

        utils.write_pickle(feature_dicts_with_attn, outpath)  # '''

In [9]:
if __name__ == "__main__":

    dataset = '/content/gdrive/MyDrive/attention_rank/P7/stackoverflow'
    text_path = dataset + '/processed_docsutf8/'
    output_path = dataset + '/processed_stackoverflow' + '/'
    save_path = output_path + 'sentence_paired_text/'

    # set which pre-trained bert is going to use
    bert_dir = "/content/gdrive/MyDrive/attention_rank/P7/pretrained_bert/orgbert/"
    bert_name = bert_dir.split('/')[1]

    files = os.listdir(text_path)
    for i, file in enumerate(files):
        files[i] = file[:-4]

    t_list = files

    # sometimes the memory will be used up,
    # change 'process_position' to where you stop
    process_position = 0
    file_list = t_list[process_position:]

    start_time = time.time()
    main(process_position)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
     0.        ]]

   [[0.03469325 0.0432214  0.04293622 ... 0.         0.
     0.        ]
    [0.04096311 0.05936005 0.0538147  ... 0.         0.
     0.        ]
    [0.03052527 0.0422221  0.04519363 ... 0.         0.
     0.        ]
    ...
    [0.04686617 0.03082473 0.04043689 ... 0.         0.
     0.        ]
    [0.04109121 0.02964382 0.04142895 ... 0.         0.
     0.        ]
    [0.04557425 0.02839915 0.0510869  ... 0.         0.
     0.        ]]]


  ...


  [[[0.04584387 0.03228277 0.0416659  ... 0.         0.
     0.        ]
    [0.05309815 0.02980642 0.03708728 ... 0.         0.
     0.        ]
    [0.04333867 0.03227634 0.03913843 ... 0.         0.
     0.        ]
    ...
    [0.03878395 0.03034984 0.04138501 ... 0.         0.
     0.        ]
    [0.04052993 0.02390626 0.04705985 ... 0.         0.
     0.        ]
    [0.04138412 0.02861743 0.03932994 ... 0.         0.
     0.        ]

In [10]:
import csv
import os
import time
import numpy as np
import pickle

In [11]:
def load_pickle(fname):
    with open(fname, "rb") as f:
        return pickle.load(f, encoding="latin1")  # add, encoding="latin1") if using python3 and downloaded data


def weights_comb(input_weights, strategy=3):
    if strategy == 1:
        comb_weight = np.array(input_weights).mean()
    elif strategy == 2:
        comb_weight = np.array(input_weights).max()
    elif strategy == 3:
        comb_weight = np.array(input_weights).sum()
    else:
        comb_weight = np.array(input_weights).prod() / np.array(input_weights).sum()
    return comb_weight


def data_iterator():
    for i, doc in enumerate(data):
        # if i % 100 == 0 or i == len(data) - 1:
        # print("{:.1f}% done".format(100.0 * (i + 1) / len(data)))
        yield doc["tokens"], np.array(doc["attns"])


def get_data_points(head_data):
    xs, ys, avgs = [], [], []
    for layer in range(12):
        for head in range(12):
            ys.append(head_data[layer, head])
            xs.append(1 + layer)
        avgs.append(head_data[layer].mean())
    return xs, ys, avgs


def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass

    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass

    return False


In [12]:
def merge_dict(x, y):
    for k, v in x.items():
        if k in y.keys():
            y[k] = y[k] + v
        else:
            y[k] = v


def map_attn(example, heads, sentence_length, layer_weight, record):
    counter_12 = 0  # check head index

    current_sum_attn = []
    for ei, (layer, head) in enumerate(heads):
        attn = example["attns"][layer][head]  # [0:sentence_length, 0:sentence_length]
        attn = np.array(attn)
        attn /= attn.sum(axis=-1, keepdims=True)  # norm each row
        attn_sum = attn.sum(axis=0, keepdims=True)  # add up 12 heads # np.shape(attn_sum) = (1,sentence length)
        words = example["tokens"]  # [0:sentence_length]
        weights_list = attn_sum[0]

        single_word_dict = {}
        for p in range(len(words)):
            hashtag_lead = words[p]
            hash_weights = [weights_list[p]]
            weight_new = weights_comb(hash_weights, 3) * layer_weight
            single_word_dict[hashtag_lead + "_" + str(p)] = weight_new  # p is the word position in sentence

        current_sum_attn.append(np.array(list(single_word_dict.values())))  # dict.values() keep the entries order
        # shape(current_sum_attn) = (12, words number), each item in it is a list of words attn in current sentence
        counter_12 += 1  # check head index

        if counter_12 % 12 == 0:  # if head number get 12, sum all 12 heads attn and output
            head = np.sum(current_sum_attn, axis=0)  # dict zip can read array, do not need to list it
            # double check
            # print(sum([item[0] for item in current_sum_attn]))
            # print(head[0])
            longer_key_list = []
            current_key_list = list(single_word_dict.keys())  # [words_positions] # dict.keys() keep the entries order
            for each_key in current_key_list:
                longer_key_list.append(each_key + "_" + str(record))  # word_positionInSentence_sentenceNumber
            current_dict = dict(zip(longer_key_list, head))  # head = word_p_s attn, heads are already summed here

            return current_dict




In [13]:
# main
start_time = time.time()

dataset = '/content/gdrive/MyDrive/attention_rank/P7/stackoverflow'
text_path = dataset + '/processed_docsutf8/'
output_path = dataset + '/processed_stackoverflow' + '/'

files = os.listdir(text_path)
for i, file in enumerate(files):
    files[i] = file[:-4]

files = files[:]

save_path = output_path + "token_attn_paired/attn/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

bert_name = "orgbert"

for n, file in enumerate(files):
    attn_extracting_dir = output_path + "sentence_paired_text/" + file + '_' + 'content' + '_attn.pkl'
    data = load_pickle(attn_extracting_dir)

    w = csv.writer(open(save_path + file + "token_attn_paired.csv", "w"))

    for r in range(len(data)):
        record = r
        sentence_length = len(data[record]['tokens'])

        # consider the 12th layer
        weight = 1
        layer = 11
        sentence_dict = map_attn(data[record],
                                 [(layer, 0), (layer, 1), (layer, 2), (layer, 3),
                                   (layer, 4), (layer, 5), (layer, 6), (layer, 7),
                                   (layer, 8), (layer, 9), (layer, 10), (layer, 11)],
                                 sentence_length, weight, record)

        # print('words:', len(document_dict))

        for k, v in sentence_dict.items():
            cut = k.find("_")  # although we do not keep word position, word position helps to keep its original order
            k_short = k[0:cut]
            w.writerow([k_short, v])

    run_time = time.time()
    print(n + 1, "th file", file, "running time", run_time - start_time)

1 th file file_12 running time 0.6147527694702148
2 th file file_84 running time 0.6605789661407471
3 th file file_19 running time 0.7500104904174805
4 th file file_97 running time 0.7657945156097412
5 th file file_96 running time 0.7812225818634033
6 th file file_25 running time 0.8121376037597656
7 th file file_58 running time 0.8524267673492432
8 th file file_67 running time 0.8794112205505371
9 th file file_43 running time 0.9291949272155762
10 th file file_81 running time 0.9492146968841553
11 th file file_89 running time 1.0037870407104492
12 th file file_28 running time 1.0841753482818604
13 th file file_56 running time 1.1270043849945068
14 th file file_24 running time 1.1387031078338623
15 th file file_47 running time 1.1718430519104004
16 th file file_8 running time 1.226508378982544
17 th file file_38 running time 1.242163896560669
18 th file file_87 running time 1.276655673980713
19 th file file_85 running time 1.29917311668396
20 th file file_61 running time 1.326360464096

In [14]:
"""Does BERT-style preprocessing of unlabeled data; heavily based on
create_pretraining_data.py in the BERT codebase. However, does not mask words
or ever use random paragraphs for the second text segment."""

import argparse
import os
import random
import time
import csv
from bert import tokenization
from configparser import ConfigParser
from swisscom_ai.research_keyphrase.preprocessing.postagging import PosTaggingCoreNLP
from swisscom_ai.research_keyphrase.model.input_representation import InputTextObj
from swisscom_ai.research_keyphrase.model.extractor import extract_candidates

In [15]:
"""
terminal command:
cd stanford-corenlp-full-2018-02-27/
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -preload tokenize,ssplit,pos -status_port 9000 -port 9000 -timeout 15000 &
"""


def load_local_corenlp_pos_tagger():
    """
    terminal command:
    cd stanford-corenlp-full-2018-02-27/
    java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -preload tokenize,ssplit,pos -status_port 9080 -port 9080 -timeout 100000 &
    """
    config_parser = ConfigParser()
    config_parser.read('/content/gdrive/MyDrive/attention_rank/P7/config.ini')
    host = config_parser.get('STANFORDCORENLPTAGGER', 'host')
    port = config_parser.get('STANFORDCORENLPTAGGER', 'port')
    return PosTaggingCoreNLP(host, port)


def prep_document(document, max_sequence_length):
    """Does BERT-style preprocessing on the provided document."""
    max_num_tokens = max_sequence_length - 3
    target_seq_length = max_num_tokens

    # We DON"T just concatenate all of the tokens from a document into a long
    # sequence and choose an arbitrary split point because this would make the
    # next sentence prediction task too easy. Instead, we split the input into
    # segments "A" and "B" based on the actual "sentences" provided by the user
    # input.
    instances = []
    current_chunk = []
    current_length = 0
    i = 0
    while i < len(document):
        segment = document[i]
        current_chunk.append(segment)
        current_length += len(segment)
        if i == len(document) - 1 or current_length >= target_seq_length:
            if current_chunk:
                a_end = 1
                if len(current_chunk) >= 2:
                    a_end = random.randint(1, len(current_chunk) - 1)

                tokens_a = []
                for j in range(a_end):
                    tokens_a.extend(current_chunk[j])

                tokens_b = []
                for j in range(a_end, len(current_chunk)):
                    tokens_b.extend(current_chunk[j])
                truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, random)

                if len(tokens_a) == 0 or len(tokens_b) == 0:
                    break
                assert len(tokens_a) >= 1
                assert len(tokens_b) >= 1

                tokens = []
                tokens.append("[CLS]")
                for token in tokens_a:
                    tokens.append(token)
                tokens.append("[SEP]")

                for token in tokens_b:
                    tokens.append(token)
                tokens.append("[SEP]")

                instances.append(tokens)

            current_chunk = []
            current_length = 0
        i += 1
    # print(instances)
    return instances

def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng):
    """Truncates a pair of sequences to a maximum sequence length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if rng.random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()

In [16]:
dataset = '/content/gdrive/MyDrive/attention_rank/P7/stackoverflow'
text_path = dataset + '/processed_docsutf8/'
output_path = dataset + '/processed_stackoverflow' + '/'
save_path = output_path + 'candidate_tokenizing/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# set which pre-trained bert is going to use
bert_dir = "/content/gdrive/MyDrive/attention_rank/P7/pretrained_bert/orgbert/"

files = os.listdir(text_path)
for i, file in enumerate(files):
    files[i] = file[:-4]

files = files[:]

start_time = time.time()

parser = argparse.ArgumentParser(description=__doc__)
parser.add_argument("--num_docs", default=1000, type=int,help="Number of documents to use (default=1000).")
parser.add_argument("--cased", default=False, action='store_true',help="Don't lowercase the input.")
parser.add_argument("-- max_sequence_length", default=512, type=int, help="Maximum input sequence length after tokenization (default=128).")
args, unknown = parser.parse_known_args()
tokenizer = tokenization.FullTokenizer(vocab_file=os.path.join(bert_dir, "vocab.txt"), do_lower_case=not args.cased)

# stanford nlp
ptagger = load_local_corenlp_pos_tagger()

df_dict = {}

In [17]:
for n, file in enumerate(files):

    random.seed(0)
    current_doc_tokens = []
    segments = []

    fp = open(text_path + file + '.txt')
    sentences = fp.read().replace('$$$$$$', ' ')

    # generate candidates
    tagged = ptagger.pos_tag_raw_text(sentences)
    text_obj = InputTextObj(tagged, 'en')
    candidates = extract_candidates(text_obj)

    w = csv.writer(open(save_path + file + "_candidate_tokenized.csv", "w"))

    # tokenize candidates
    raw_lines = []
    for l, line in enumerate(candidates):
        raw_lines.append(line)
        line = tokenization.convert_to_unicode(line).strip()
        if not line:  # line is empty, deal the 2 segments in the [current doc tokens]
            if current_doc_tokens:
                for segment in prep_document(
                        current_doc_tokens, args.max_sequence_length):
                    if (len(segment) - 1) / 2 != segment.index("[SEP]"):
                        cline = raw_lines[l - 1].replace("),", "),$$$$$").split("$$$$$")
                    segments.append(segment)
                    if len(segments) >= args.num_docs:
                        break
                if len(segments) >= args.num_docs:
                    break

            current_doc_tokens = []  # clean up [current doc tokens]

        tokens = tokenizer.tokenize(line)
        # print(line, tokens)
        w.writerow([line, tokens])

        if tokens:  # if line is not empty, add lines to [current doc tokens]
            current_doc_tokens.append(tokens)
    
    # get tf
    tf_dict = {}
    for item in candidates:
        if item in tf_dict:
            tf_dict[item] += 1
        else:
            tf_dict[item] = 1

    w0 = csv.writer(open(save_path + file + '_candidate_tf.csv', "a"))
    # get df
    for k, v in sorted(tf_dict.items(), key=lambda item: item[1], reverse=True):
        w0.writerow([k, v])
        if k in df_dict:
            df_dict[k] += 1
        else:
            df_dict[k] = 1
    
    run_time = time.time()
    print(n, "th file", file, "running time", run_time - start_time)

0 th file file_12 running time 6.039825201034546
1 th file file_84 running time 6.385778188705444
2 th file file_19 running time 6.480538606643677
3 th file file_97 running time 6.541836977005005
4 th file file_96 running time 6.612131118774414
5 th file file_25 running time 6.682356357574463
6 th file file_58 running time 6.746665000915527
7 th file file_67 running time 6.810412883758545
8 th file file_43 running time 6.8753416538238525
9 th file file_81 running time 6.931495904922485
10 th file file_89 running time 7.000582695007324
11 th file file_28 running time 7.072190523147583
12 th file file_56 running time 7.133959770202637
13 th file file_24 running time 7.200263738632202
14 th file file_47 running time 7.280321359634399
15 th file file_8 running time 7.361666202545166
16 th file file_38 running time 7.41310453414917
17 th file file_87 running time 7.4678943157196045
18 th file file_85 running time 7.521209001541138
19 th file file_61 running time 7.577955722808838
20 th file

In [18]:
##w1 = csv.writer(open(save_path + projet + '_candidate_df.csv', "a")) --- "project is not defined!!"##
w1 = csv.writer(open(save_path + file + '_candidate_df.csv', "a"))
for k, v in sorted(df_dict.items(), key=lambda item:item[1], reverse=True):
    w1.writerow([k, v])


In [19]:
import os
import time
import csv


"""pair candidates and their accumulated self-attention"""


dataset = '/content/gdrive/MyDrive/attention_rank/P7/stackoverflow'
doc_path = dataset + '/processed_docsutf8/'
output_path = dataset + '/processed_stackoverflow' + '/'
candidate_token_path = output_path + 'candidate_tokenizing/'
token_attn_path = output_path + 'token_attn_paired/attn/'

save_path = output_path + 'candidate_attn_paired/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

files = os.listdir(doc_path)
for i, file in enumerate(files):
    files[i] = file[:-4]

files = files[:]

start_time = time.time()

for n, file in enumerate(files):

    w = csv.writer(open(save_path + file + "_attn_paired.csv", "w"))

    # read token attn to list
    token_list = []
    attn_list = []
    with open(token_attn_path + file + "token_attn_paired.csv", newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        for row in spamreader:
            k = row[0]
            v = float(row[1])
            token_list.append(k)
            attn_list.append(v)

    # read candidate tokens to dict
    candidate_token_dict = {}
    with open(candidate_token_path + file + "_candidate_tokenized.csv", newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        for row in spamreader:
            k = row[0]
            v = row[1][2:-2].split("', '")
            candidate_token_dict[k] = v

    # candidate attn pairing
    candidate_attn_dict = {}
    for k, v in candidate_token_dict.items():
        window = len(v)
        matched = []
        for t, token in enumerate(token_list):
            if token_list[t:t+window] == v:
                local_attn = sum(attn_list[t:t+window])
                if k in candidate_attn_dict.keys():
                    candidate_attn_dict[k] += local_attn
                else:
                    candidate_attn_dict[k] = local_attn

    for k, v in candidate_attn_dict.items():
        w.writerow([k, v])

    run_time = time.time()
    print(n, "th file", file, "running time", run_time - start_time)

0 th file file_12 running time 0.00617671012878418
1 th file file_84 running time 0.013963699340820312
2 th file file_19 running time 0.030013084411621094
3 th file file_97 running time 0.03688216209411621
4 th file file_96 running time 0.06551623344421387
5 th file file_25 running time 0.07246756553649902
6 th file file_58 running time 0.07891607284545898
7 th file file_67 running time 0.08503198623657227
8 th file file_43 running time 0.09084939956665039
9 th file file_81 running time 0.09534740447998047
10 th file file_89 running time 0.10298609733581543
11 th file file_28 running time 0.11486983299255371
12 th file file_56 running time 0.12126684188842773
13 th file file_24 running time 0.12629389762878418
14 th file file_47 running time 0.13247203826904297
15 th file file_8 running time 0.1405010223388672
16 th file file_38 running time 0.14609575271606445
17 th file file_87 running time 0.15119528770446777
18 th file file_85 running time 0.15640687942504883
19 th file file_61 run